# 

In this, we will be working on an image classification task (codename `pnp`) using the transfer learning technique.
The task objective is to determine whether an image contains a person (`pnp` stands for person / non-person) -- a binary classification task.

## Dataset
* Dataset contains 80K images with known labels (for model development), and 20K images with unknown labels (for scoring).
* Dataset has been created from a subset of COCO Dataset, and so all copyrights belong to the original authors: https://cocodataset.org/#termsofuse
* Images have been rescaled and padded to be of shape (224, 224, 3).

While it's possible to create a new model architecture and train a model specifically for this task, that would be expensive in terms of time and cloud resources.
Instead, in this assignment, you will be re-using an pre-trained model's architecture and parameters to save time and cloud resources.

## MobileNet Architecture
* The pre-trained model's name is MobileNetV2: https://arxiv.org/pdf/1801.04381.pdf
* MobileNet is a relatively small network that is designed for usage on mobile devices with limited compute and storage resource.
* It's a great choice for this assignment, since this network can be relatively quickly processed with a single GPU.

## MobileNet Parameters
* Keras provides network architecture and pre-trained parameters: https://keras.io/api/applications/mobilenet/#mobilenetv2-function
* The pre-trained parameters come from the ImageNet 1000-class task, which does not include a person label.
* The lower part of the network can be reused due to the shared hierarchy of visual information..


In [7]:
!ls pnp_dataset.zip || (aws s3 cp s3://danylo-ucla/pnp_dataset.zip ./)

pnp_dataset.zip


In [8]:
import matplotlib.pyplot as plt

In [9]:
# Run this cell to configure Tensorflow to use your GPU
import tensorflow as tf
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    print(gpu)
    tf.config.experimental.set_memory_growth(gpu, True)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


In [10]:
from collegium.m02_cnn.utils.pnp_dataset import build_dataset

# See Tensorflow Dataset API for details
# https://www.tensorflow.org/api_docs/python/tf/data/Dataset
train_ds = build_dataset('train', include_labels=True)
score_ds = build_dataset('score', include_labels=False)
train_ds, score_ds

(<_FlatMapDataset element_spec=(TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>,
 <_FlatMapDataset element_spec=TensorSpec(shape=(224, 224, 3), dtype=tf.float32, name=None)>)

In [12]:
# https://arxiv.org/pdf/1801.04381.pdf
# https://keras.io/api/applications/mobilenet/#mobilenetv2-function
import keras_cv
mobile_net = keras_cv.models.MobileNetV3Backbone.from_preset(
    "mobilenet_v3_small_imagenet",
    input_shape=(224, 224, 3)
)

100%|██████████| 1.85k/1.85k [00:00<00:00, 4.09MB/s]


100%|██████████| 3.95M/3.95M [00:00<00:00, 6.03MB/s]


In [13]:
# Freezes the parameters of the MobileNet layers, so they will not update during training.
# These parameters are initialized to a pre-trained snapshot using the ImagetNet dataset.
mobile_net.trainable = False

In [14]:
# mobile_net.summary()

In [15]:
model = tf.keras.models.Sequential([
    mobile_net,
    tf.keras.layers.AvgPool2D(pool_size=(5, 5)),
    tf.keras.layers.Flatten(),
    
    # Add a Dense layer with more units
    tf.keras.layers.Dense(units=128, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.015)),
    
    # Add a Dropout layer for regularization
    tf.keras.layers.Dropout(0.5),
    
    # Add Batch Normalization for improved convergence
    tf.keras.layers.BatchNormalization(),
    
    # Final Output Layer (binary classification)
    tf.keras.layers.Dense(units=1, activation='sigmoid')  # Sigmoid for binary classification
])


In [16]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobile_net_v3_small_backbone    │ (None, 7, 7, 576)      │       939,120 │
│ (MobileNetV3Backbone)           │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 1, 1, 576)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 576)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,013,617 (3.87 MB)

 Trainable params: 74,241 (290.00 KB)

 Non-trainable params: 939,376 (3.58 MB)

In [17]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),  # Use Adam optimizer
    loss='binary_crossentropy',
    metrics=[
        tf.keras.metrics.BinaryAccuracy(
            name="binary_accuracy",
            threshold=0.5
        )
    ]
)

In [18]:

# Define the split percentages and shuffle the dataset
total_size = 80000  # Replace with the actual number of examples in the dataset
validation_split = 0.25  # 20% for validation
train_size = int(total_size * (1 - validation_split))

# Shuffle the dataset (shuffling is important before splitting)
train_ds = train_ds.shuffle(buffer_size=total_size, seed=42)

# Split the dataset into training and validation sets
train_ds_split = train_ds.take(train_size)  # First part for training
val_ds_split = train_ds.skip(train_size)    # Remaining part for validation

# Batch the datasets (keep the data in TensorFlow's Dataset format)
batch_size = 512
train_ds_split = train_ds_split.batch(batch_size)
val_ds_split = val_ds_split.batch(batch_size)

# Prefetch the dataset for better performance
train_ds_split = train_ds_split.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
val_ds_split = val_ds_split.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)


early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='binary_accuracy', 
    patience=3,  # Stop training if val_loss doesn't improve for 3 consecutive epochs
    restore_best_weights=True
)

history = model.fit(
    train_ds_split,
    validation_data=val_ds_split,
    epochs=25,  # Increase number of epochs
    callbacks=[early_stopping]
)




Epoch 1/25


I0000 00:00:1728877265.978116    2287 service.cc:146] XLA service 0x7fd68000ea80 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728877265.978145    2287 service.cc:154]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1728877280.380034    2287 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


    118/Unknown 226s 195ms/step - binary_accuracy: 0.6050 - loss: 3.7350

/app/conda/envs/collegium/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


118/118 ━━━━━━━━━━━━━━━━━━━━ 401s 2s/step - binary_accuracy: 0.6057 - loss: 3.7328 - val_binary_accuracy: 0.8094 - val_loss: 2.9609
Epoch 2/25
118/118 ━━━━━━━━━━━━━━━━━━━━ 351s 2s/step - binary_accuracy: 0.7773 - loss: 2.8799 - val_binary_accuracy: 0.8238 - val_loss: 2.3894
Epoch 3/25
118/118 ━━━━━━━━━━━━━━━━━━━━ 352s 2s/step - binary_accuracy: 0.7985 - loss: 2.3218 - val_binary_accuracy: 0.8267 - val_loss: 1.9426
Epoch 4/25
118/118 ━━━━━━━━━━━━━━━━━━━━ 352s 2s/step - binary_accuracy: 0.8107 - loss: 1.8889 - val_binary_accuracy: 0.8392 - val_loss: 1.5868
Epoch 5/25
118/118 ━━━━━━━━━━━━━━━━━━━━ 349s 2s/step - binary_accuracy: 0.8198 - loss: 1.5538 - val_binary_accuracy: 0.8425 - val_loss: 1.3179
Epoch 6/25
118/118 ━━━━━━━━━━━━━━━━━━━━ 349s 2s/step - binary_accuracy: 0.8265 - loss: 1.2966 - val_binary_accuracy: 0.8467 - val_loss: 1.1085
Epoch 7/25
118/118 ━━━━━━━━━━━━━━━━━━━━ 352s 2s/step - binary_accuracy: 0.8361 - loss: 1.0895 - val_binary_accuracy: 0.8464 - val_loss: 0.9557
Epoch 8/25

In [20]:
import os 
import pandas as pd

model_dir = 'pnp_model'

os.makedirs(model_dir, exist_ok=True)

# Once you are ready to make the graded submission,
# run the model on the score dataset.
score_y_hat = pd.DataFrame(
    model.predict(score_ds.batch(batch_size)),
    # This is needed to save the file in Parquet format.
    columns=['score']
)

# Now save it to disc as a Parquet file.
score_y_hat.to_parquet(f'{model_dir}/score_y_hat.parquet')
assert score_y_hat.shape == (20000, 1)

# Next, let's save the model's definition.
import json
with open(f'{model_dir}/model_architecture.json', 'w') as f:
    f.write(json.dumps(json.loads(model.to_json()), indent=True))

# Finally, let's save the learned parameters.
tf.keras.models.save_model(model, f'{model_dir}/model.keras')

# You now have the following files to be uploaded to Moodle:
# 1. This notebook and any other Python code you used to train the final model.
# 2. model_architecture.json -- the model's definition
# 3. model.keras -- the model's trained parameters
# 4. score_y_hat.parquet - the model's output on the score dataset

40/40 ━━━━━━━━━━━━━━━━━━━━ 48s 1s/step
